In [1]:
import scvi
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import sys

/home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/python3.11/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/python3.11/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
condition_key = 'embryo_id' #change name?
label_key = 'cellcluster_moscot' #to celltypes?
adata_path = f'/home/icb/jonas.flor/gastrulation_atlas/scvi/training/1M/2k_genes/2/4196'
metrics_path = f'/home/icb/jonas.flor/gastrulation_atlas/scvi/training/1M/2k_genes/2/4196/metrics'

if sys.argv[1]=='10k':
    n_obs=10000
elif sys.argv[1]=='100k':
    n_obs=100000
elif sys.argv[1]=='500k':
    n_obs=500000
elif sys.argv[1]=='1M':
    n_obs=1000000

if sys.argv[1]!='all':
    adata = sc.read_h5ad(f'/lustre/groups/ml01/workspace/monge_velo/data/mouse_gastrulation_atlas/training/adata_1M.h5ad')
else:
    adata = sc.read_h5ad('/lustre/groups/ml01/workspace/monge_velo/data/mouse_gastrulation_atlas/gastrulation_atlas_pca.h5ad')
adata

AnnData object with n_obs × n_vars = 1000000 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'batch', 'cell_cluster', 'celltype', 'cellcluster_moscot'
    var: 'gene_type', 'celltype', 'chr'
    uns: 'neighbors'
    obsm: 'X_pca'
    obsp: 'connectivities', 'distances'

In [3]:
obsp = adata.obsp
highly_variable = adata.var_names

if sys.argv[2]=='2k_genes':
    adata_copy = adata.copy()
    sc.pp.normalize_total(adata_copy)
    sc.pp.log1p(adata_copy)
    sc.pp.highly_variable_genes(
        adata_copy,
        n_top_genes=2000,
        batch_key=condition_key,
    )
    highly_variable = adata_copy.var["highly_variable"]
    del adata_copy
elif sys.argv[2]=='10k_genes':
    adata_copy = adata.copy()
    sc.pp.normalize_total(adata_copy)
    sc.pp.log1p(adata_copy)
    sc.pp.highly_variable_genes(
        adata_copy,
        n_top_genes=10000,
        batch_key=condition_key,
    )
    highly_variable = adata_copy.var["highly_variable"]
    del adata_copy

adata = ad.concat([
    sc.read_h5ad('/lustre/groups/ml01/workspace/monge_velo/data/mouse_gastrulation_atlas/training/validation.h5ad'),
    adata
])
adata._inplace_subset_var(highly_variable)

var = adata.var #otherwise not subsetted to hgv genes

In [4]:
adata.X[:10,:10].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 4., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [12]:
scvi.model.SCVI.setup_anndata(adata, batch_key=condition_key)
vae = scvi.model.SCVI(
    adata,
    n_layers=2,
    n_hidden=8192,
    n_latent=50
)
vae.train(plan_kwargs={'lr': 1e-4})
#vae.save(adata_path, overwrite=True)

/home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/p ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/p ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/7:   0%|                                                                                                                                                                                                               | 0/7 [00:00<?, ?it/s]

/home/icb/jonas.flor/mambaforge/envs/scvi_test/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
adata = adata[100000:]
adata.var = var
adata.obsp = obsp

adata.write(f'{adata_path}/unintegrated_adata.h5ad')
del adata

#training metrics
pd.DataFrame(
    np.concatenate([vae.history['train_loss_epoch'], vae.history['validation_loss']], axis=1), columns=['train_loss', 'val_loss']
).to_csv(
    f'{metrics_path}/loss.csv'
)

pd.DataFrame(
    np.concatenate([vae.history['elbo_train'], vae.history['elbo_validation']], axis=1), columns=['train_elbo', 'val_elbo']
).to_csv(
    f'{metrics_path}/elbo.csv'
)

pd.DataFrame(
    np.concatenate([vae.history['reconstruction_loss_train'], vae.history['reconstruction_loss_validation']], axis=1), columns=['train_recon_loss', 'val_recon_loss']
).to_csv(
    f'{metrics_path}/reconstr_loss.csv'
)